# Upper Green PEST Pilot Point Setup

In [3]:
import sys
import numpy as np
import pandas as pd
import flopy as flopy
import pyemu
import shapefile #the pyshp module

flopy is installed in C:\Users\seonggyu.park\Miniconda3\envs\sm_pest\lib\site-packages\flopy


# <span style="color:yellow">1. Change Initial K using K shapefile</span>.

There are multiple approaches to implementing pilot points with PEST++.  

In this class, we will use some kick-ass pyemu sweetness

### 1.1. Set up zones for where pilot points will be interpolated

We can have pilot point networks in multiple zones. In this case, we will make a simple zone file using `IBOUND` such that all active cells are in the same interpolation zone.

In [4]:
wd = 'D:/Projects/Watersheds/Green/Analysis/APEX-MODFLOWs/calibrations/gr_210707/gr_012721/APEX-MODFLOW'
pstfile = "green_pest.pst"
os.chdir(wd)

In [5]:
# load the pre-constructed pst
pst = pyemu.Pst(os.path.join(wd, pstfile))

In [6]:
par = pst.parameter_data

In [14]:
shp = 'D:/Projects/Watersheds/Green/GIS/hk_initial_f.shp'

In [15]:
#read file, parse out the records and shapes
sf = shapefile.Reader(shp)
#grab the shapefile's field names (omit the first psuedo field)
fields = [x[0] for x in sf.fields][1:]
records = sf.records()
shps = [s.points for s in sf.shapes()]
#write the records into a dataframe
shapefile_dataframe = pd.DataFrame(columns=fields, data=records)
#add the coordinate data to a column called "coords"
shapefile_dataframe = shapefile_dataframe.assign(coords=shps)
pp_df = shapefile_dataframe.sort_values(by=['name'])
hk_df = pp_df[['parnme', 'K_m_day_']]
hk_df.index = hk_df.parnme
hk_df

,parnme,K_m_day_
parnme,,
hk0000,hk0000,26.092383
hk0001,hk0001,0.000027
hk0002,hk0002,1.369348
hk0003,hk0003,0.273221
hk0004,hk0004,26.092383
...,...,...
hk0105,hk0105,0.000545
hk0106,hk0106,0.000027
hk0107,hk0107,0.273221


In [7]:
k01 = [87, 88, 92, 93, 94, 95]
k01 = ['hk{:04d}'.format(i) for i in k01]
k01

['hk0087', 'hk0088', 'hk0092', 'hk0093', 'hk0094', 'hk0095']

In [9]:
for i in par.parnme:
    for j in k01:
        if i == j:
            par.loc[i, 'parval1'] = 0.1
            par.loc[i, 'parlbnd'] = 0.1* 0.01
            par.loc[i, 'parubnd'] = 0.1* 100
            

In [16]:
# for i in par.parnme:
#     for j in hk_df.parnme:
#         if i == j:
#             if hk_df.loc[j, 'K_m_day_'] < 0.01:
#                 hk_df.loc[j, 'K_m_day_'] = 1
#             if hk_df.loc[j, 'K_m_day_'] > 26:
#                 hk_df.loc[j, 'K_m_day_'] = 30
#             par.loc[i, 'parval1'] = hk_df.loc[j, 'K_m_day_']
#             par.loc[i, 'parlbnd'] = hk_df.loc[j, 'K_m_day_'] * 0.01
#             par.loc[i, 'parubnd'] = hk_df.loc[j, 'K_m_day_'] * 100
            

C:\Users\seonggyu.park\Miniconda3\envs\sm_pest\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [10]:
for i in range(len(par)):
    if par.iloc[i, 6] == 'rivbot':
        par.iloc[i, 3] = 2.0001   # initial    
        par.iloc[i, 4] = 0.1   # lower
        par.iloc[i, 5] = 4   # upper
        par.iloc[i, 8] = -2   # offset
    elif par.iloc[i, 6] == 'sy':
        par.iloc[i, 3] = 1.000000e-01 
        par.iloc[i, 4] = 1.000000e-03
        par.iloc[i, 5] = 0.500000e+00   

In [11]:
pst.parameter_data = par
pst.write('green_pest_3rd.pst')

noptmax:0, npar_adj:251, nnz_obs:698
